# This is an old version! Use BuildTrainTestData.ipynb instead

# Building the feature table

- Features: The features are the B-Point locations extracted by the different B-Point algorithms
- Target variable: Manually labeled reference B-Point location
- Samples: Data on the heartbeat level

### Setup and helper functions

In [2]:
from pathlib import Path

import pandas as pd
from sklearn.model_selection import GroupShuffleSplit

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [3]:
root_path = Path("../../")

In [4]:
result_path = root_path.joinpath("results")
result_path

WindowsPath('../../results')

In [5]:
empkins_pivot_df = pd.read_csv(
    result_path.joinpath("pivot_dataframe_b_point/empkins_pivot_dataframe_b_point_algos.csv")
).drop(columns="Unnamed: 0")
guardian_pivot_df = pd.read_csv(
    result_path.joinpath("pivot_dataframe_b_point/guardian_pivot_dataframe_b_point_algos.csv")
).drop(columns="Unnamed: 0")

### Generate train and test data for the empkins dataset

In [6]:
group_split_empkins = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=1)
train_idx_empkins, test_idx_empkins = next(
    group_split_empkins.split(empkins_pivot_df, groups=empkins_pivot_df["participant"])
)
train_data_empkins = empkins_pivot_df.iloc[train_idx_empkins]
test_data_empkins = empkins_pivot_df.iloc[test_idx_empkins]

In [7]:
train_data_empkins["participant"].unique()

array(['VP_001', 'VP_002', 'VP_003', 'VP_005', 'VP_020', 'VP_026',
       'VP_027', 'VP_028', 'VP_029', 'VP_030', 'VP_031', 'VP_032'],
      dtype=object)

In [8]:
test_data_empkins["participant"].unique()

array(['VP_004', 'VP_022', 'VP_023'], dtype=object)

In [9]:
(test_data_empkins["participant"].isin(train_data_empkins["participant"]).any()) is True

False

### Generate train and test data for the guardian dataset

In [10]:
group_split_guardian = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=1)
train_idx_guardian, test_idx_guardian = next(
    group_split_guardian.split(guardian_pivot_df, groups=guardian_pivot_df["participant"])
)
train_data_guardian = guardian_pivot_df.iloc[train_idx_guardian]
test_data_guardian = guardian_pivot_df.iloc[test_idx_guardian]

In [11]:
train_data_guardian["participant"].unique()

array(['GDN0005', 'GDN0006', 'GDN0007', 'GDN0009', 'GDN0010', 'GDN0011',
       'GDN0012', 'GDN0013', 'GDN0014', 'GDN0016', 'GDN0017', 'GDN0018',
       'GDN0021', 'GDN0022', 'GDN0023', 'GDN0025', 'GDN0028', 'GDN0029',
       'GDN0030'], dtype=object)

In [12]:
test_data_guardian["participant"].unique()

array(['GDN0008', 'GDN0019', 'GDN0020', 'GDN0024', 'GDN0027'],
      dtype=object)

In [13]:
(test_data_guardian["participant"].isin(train_data_guardian["participant"]).any()) is True

False

### Combine train and test data of both datasets

In [14]:
train_df = pd.concat([train_data_empkins, train_data_guardian], ignore_index=True)
train_df = train_df.drop(columns=["participant", "condition", "phase", "heartbeat_idreference"])
train_df

,b_point_samplereference,multiple-conditions_autoregression,multiple-conditions_linear-interpolation,multiple-conditions_none,second-derivative_autoregression,second-derivative_linear-interpolation,second-derivative_none,straight-line_autoregression,straight-line_linear-interpolation,straight-line_none,third-derivative_autoregression,third-derivative_linear-interpolation,third-derivative_none
0,1074,1231.0,1238.0,1063.0,1063.0,1063.0,1063.0,1116.0,1116.0,1116.0,1134.0,1134.0,1134.0
1,1849,1888.0,1889.0,1816.0,1829.0,1829.0,1829.0,1853.0,1853.0,1853.0,1872.0,1879.0,1859.0
2,2518,2517.0,2517.0,2517.0,2508.0,2508.0,2508.0,2542.0,2542.0,2542.0,2577.0,2577.0,2577.0
3,3252,3247.0,3247.0,3247.0,3228.0,3228.0,3228.0,3270.0,3270.0,3270.0,3295.0,3295.0,3295.0
4,3933,3932.0,3932.0,3932.0,3917.0,3917.0,3917.0,3940.0,3940.0,3940.0,3949.0,3956.0,3857.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9142,27597,27582.0,27582.0,27582.0,27596.0,27596.0,27596.0,27603.0,27603.0,27603.0,27606.0,27606.0,27606.0
9143,28086,28074.0,28074.0,28074.0,28085.0,28085.0,28085.0,28091.0,28091.0,28091.0,28092.0,28092.0,28092.0
9144,28581,28595.0,28595.0,28595.0,28602.0,28602.0,28602.0,28592.0,28592.0,28592.0,28612.0,28612.0,28612.0
9145,29061,29058.0,29058.0,29058.0,29089.0,29089.0,29089.0,29091.0,29091.0,29091.0,29097.0,29097.0,29097.0


In [15]:
print(
    f"The length of the combined train data {train_df.shape[0]} should match the summation of the train data from the guardian and empkins datasets {train_data_empkins.shape[0] + train_data_guardian.shape[0]}"
)

The length of the combined train data 9147 should match the summation of the train data from the guardian and empkins datasets 9147


In [16]:
test_df = pd.concat([test_data_empkins, test_data_guardian], ignore_index=True)
test_df = test_df.drop(columns=["participant", "condition", "phase", "heartbeat_idreference"])
test_df

,b_point_samplereference,multiple-conditions_autoregression,multiple-conditions_linear-interpolation,multiple-conditions_none,second-derivative_autoregression,second-derivative_linear-interpolation,second-derivative_none,straight-line_autoregression,straight-line_linear-interpolation,straight-line_none,third-derivative_autoregression,third-derivative_linear-interpolation,third-derivative_none
0,593,564.0,564.0,NaN,585.0,585.0,585.0,596.0,596.0,596.0,603.0,603.0,603.0
1,1231,1207.0,1208.0,NaN,1228.0,1228.0,1223.0,1238.0,1238.0,1238.0,1244.0,1245.0,1240.0
2,1896,1916.0,1916.0,1916.0,1937.0,1937.0,1937.0,1944.0,1942.0,1902.0,1950.0,1951.0,1900.0
3,2790,2811.0,2811.0,2811.0,2826.0,2826.0,2826.0,2835.0,2835.0,2835.0,2845.0,2845.0,2845.0
4,3740,3712.0,3729.0,3683.0,3735.0,3735.0,3735.0,3748.0,3748.0,3748.0,3753.0,3753.0,3753.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2087,27246,27234.0,27234.0,27234.0,27264.0,27264.0,27264.0,27252.0,27252.0,27252.0,27252.0,27252.0,27252.0
2088,27715,27714.0,27714.0,27714.0,27710.0,27710.0,27710.0,27725.0,27725.0,27725.0,27727.0,27727.0,27727.0
2089,28208,28207.0,28207.0,28207.0,28211.0,28214.0,28193.0,28226.0,28226.0,28226.0,28249.0,28249.0,28249.0
2090,28773,28772.0,28772.0,28772.0,28767.0,28767.0,28767.0,28777.0,28777.0,28777.0,28775.0,28775.0,28775.0


In [17]:
print(
    f"The length of the combined test data {test_df.shape[0]} should match the summation of the test data from the guardian and empkins datasets {test_data_empkins.shape[0] + test_data_guardian.shape[0]}"
)

The length of the combined test data 2092 should match the summation of the test data from the guardian and empkins datasets 2092


### Split the data in actual data and target variable

#### Extract and save the target variables

In [18]:
train_target = train_df["b_point_samplereference"]
test_target = test_df["b_point_samplereference"]
print(f"Train target:\n {train_target}\n Test target:\n {test_target}")

Train target:
 0        1074
1        1849
2        2518
3        3252
4        3933
        ...  
9142    27597
9143    28086
9144    28581
9145    29061
9146    29594
Name: b_point_samplereference, Length: 9147, dtype: int64
 Test target:
 0         593
1        1231
2        1896
3        2790
4        3740
        ...  
2087    27246
2088    27715
2089    28208
2090    28773
2091    29320
Name: b_point_samplereference, Length: 2092, dtype: int64


In [19]:
# train_target.to_csv(result_path.joinpath("train_test_data/train_target.csv"))
# test_target.to_csv(result_path.joinpath("train_test_data/test_target.csv"))

#### Extract and save the train data

In [20]:
train_data = train_df.drop(columns=["b_point_samplereference"])
test_data = test_df.drop(columns=["b_point_samplereference"])

In [22]:
train_data

,multiple-conditions_autoregression,multiple-conditions_linear-interpolation,multiple-conditions_none,second-derivative_autoregression,second-derivative_linear-interpolation,second-derivative_none,straight-line_autoregression,straight-line_linear-interpolation,straight-line_none,third-derivative_autoregression,third-derivative_linear-interpolation,third-derivative_none
0,1231.0,1238.0,1063.0,1063.0,1063.0,1063.0,1116.0,1116.0,1116.0,1134.0,1134.0,1134.0
1,1888.0,1889.0,1816.0,1829.0,1829.0,1829.0,1853.0,1853.0,1853.0,1872.0,1879.0,1859.0
2,2517.0,2517.0,2517.0,2508.0,2508.0,2508.0,2542.0,2542.0,2542.0,2577.0,2577.0,2577.0
3,3247.0,3247.0,3247.0,3228.0,3228.0,3228.0,3270.0,3270.0,3270.0,3295.0,3295.0,3295.0
4,3932.0,3932.0,3932.0,3917.0,3917.0,3917.0,3940.0,3940.0,3940.0,3949.0,3956.0,3857.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9142,27582.0,27582.0,27582.0,27596.0,27596.0,27596.0,27603.0,27603.0,27603.0,27606.0,27606.0,27606.0
9143,28074.0,28074.0,28074.0,28085.0,28085.0,28085.0,28091.0,28091.0,28091.0,28092.0,28092.0,28092.0
9144,28595.0,28595.0,28595.0,28602.0,28602.0,28602.0,28592.0,28592.0,28592.0,28612.0,28612.0,28612.0
9145,29058.0,29058.0,29058.0,29089.0,29089.0,29089.0,29091.0,29091.0,29091.0,29097.0,29097.0,29097.0


In [24]:
nan_rows = train_data[train_data.isnull().any(axis=1)]
nan_rows

,multiple-conditions_autoregression,multiple-conditions_linear-interpolation,multiple-conditions_none,second-derivative_autoregression,second-derivative_linear-interpolation,second-derivative_none,straight-line_autoregression,straight-line_linear-interpolation,straight-line_none,third-derivative_autoregression,third-derivative_linear-interpolation,third-derivative_none
11,8984.0,8984.0,8984.0,9061.0,9051.0,NaN,9018.0,9018.0,9018.0,9036.0,9036.0,9036.0
27,20282.0,20282.0,20282.0,20312.0,20316.0,NaN,20315.0,20315.0,20315.0,20348.0,20348.0,20348.0
38,28111.0,28111.0,28111.0,28156.0,28146.0,NaN,28166.0,28166.0,28166.0,28183.0,28183.0,28183.0
41,829.0,829.0,829.0,864.0,864.0,NaN,892.0,892.0,892.0,919.0,919.0,919.0
42,1493.0,1493.0,1493.0,1508.0,1506.0,NaN,1542.0,1546.0,1537.0,1564.0,1564.0,1564.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9035,583.0,584.0,NaN,589.0,589.0,589.0,595.0,595.0,595.0,598.0,598.0,598.0
9036,1107.0,1110.0,NaN,1120.0,1117.0,1101.0,1122.0,1122.0,1122.0,1129.0,1129.0,1129.0
9065,16447.0,16445.0,NaN,16469.0,16469.0,16469.0,16476.0,16476.0,16476.0,16478.0,16478.0,16478.0
9110,11090.0,11092.0,NaN,11130.0,11130.0,11130.0,11104.0,11105.0,11081.0,11136.0,11136.0,11136.0


In [66]:
test_data

,multiple-conditions_autoregression,multiple-conditions_linear-interpolation,multiple-conditions_none,second-derivative_autoregression,second-derivative_linear-interpolation,second-derivative_none,straight-line_autoregression,straight-line_linear-interpolation,straight-line_none,third-derivative_autoregression,third-derivative_linear-interpolation,third-derivative_none
0,564.0,564.0,NaN,585.0,585.0,585.0,596.0,596.0,596.0,603.0,603.0,603.0
1,1207.0,1208.0,NaN,1228.0,1228.0,1223.0,1238.0,1238.0,1238.0,1244.0,1245.0,1240.0
2,1916.0,1916.0,1916.0,1937.0,1937.0,1937.0,1944.0,1942.0,1902.0,1950.0,1951.0,1900.0
3,2811.0,2811.0,2811.0,2826.0,2826.0,2826.0,2835.0,2835.0,2835.0,2845.0,2845.0,2845.0
4,3712.0,3729.0,3683.0,3735.0,3735.0,3735.0,3748.0,3748.0,3748.0,3753.0,3753.0,3753.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2087,27234.0,27234.0,27234.0,27264.0,27264.0,27264.0,27252.0,27252.0,27252.0,27252.0,27252.0,27252.0
2088,27714.0,27714.0,27714.0,27710.0,27710.0,27710.0,27725.0,27725.0,27725.0,27727.0,27727.0,27727.0
2089,28207.0,28207.0,28207.0,28211.0,28214.0,28193.0,28226.0,28226.0,28226.0,28249.0,28249.0,28249.0
2090,28772.0,28772.0,28772.0,28767.0,28767.0,28767.0,28777.0,28777.0,28777.0,28775.0,28775.0,28775.0


In [68]:
# train_data.to_csv(result_path.joinpath("train_test_data/train_data.csv"))
# test_data.to_csv(result_path.joinpath("train_test_data/test_data.csv"))